In [1]:
import pandas as pd
import numpy as np
import math
import re
import time

In [2]:
import datetime
import dateutil.parser as dparser

In [3]:
import datetime
import dateutil.parser as dparser

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [5]:
traindata = pd.read_csv('train.csv')

In [6]:
testdata = pd.read_csv('test.csv')

In [7]:
default_date = datetime.datetime(2000,1,1)

In [8]:
def is_freetonumber(is_free):
    if(is_free==False):
        return 0
    if(is_free==True):
        return 1

In [9]:
def stringtolist(data):
    ans = []
    for i in range(len(data)):
        temp_list = data[i].split(',')
        for j in range(len(temp_list)):
            temp = temp_list[j]
            ans.append(temp)
    return ans

In [10]:
def texttodictionary(column):
    c_list = stringtolist(column)
    dictionary = set(c_list)
    return dictionary

In [11]:
def texttoonehot(dictionary,column,data):
    for i in dictionary:
        ans = []
        for j in range(len(column)):
            text_list = column[j].split(',')
            temp = 0 
            for k in range(len(text_list)):
                if(text_list[k] == i):
                    ans.append(1)
                    temp = 1
                    break
            if(temp == 0):
                ans.append(0)
        n_column = {i:ans}
        n_column = pd.DataFrame(n_column)
        data = pd.concat([data,n_column],axis=1) 
    return data
#convert string like tages,categories to one-hot code,connect them with the original dataframe

In [12]:
def stringtodatetime(string):
    if(type(string) == str):
        cur_date = dparser.parse(string,fuzzy=True)
        return ((cur_date-default_date).days)
#convert date to the number of days from default date

In [13]:
for i in range(len(traindata.is_free)):
    traindata.is_free.loc[i] = is_freetonumber(traindata.is_free[i])

C:\Users\lenovo\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
for i in range(len(traindata.purchase_date)):
    traindata.purchase_date.loc[i] = stringtodatetime(traindata.purchase_date[i])
    traindata.release_date.loc[i] = stringtodatetime(traindata.release_date[i])
aver_purchase=0
aver_release=0
for i in range(len(traindata.release_date)):
    if(traindata.purchase_date[i] != None):
        aver_purchase += traindata.purchase_date[i]
    if(traindata.release_date[i] != None):
        aver_release += traindata.release_date[i]
aver_purchase = int(aver_purchase/len(traindata.release_date))
aver_release = int(aver_release/len(traindata.release_date))
#fill None with average number
for i in range(len(traindata.release_date)):
    if(traindata.purchase_date[i] == None):
        traindata.purchase_date.loc[i] = aver_purchase
    if(traindata.release_date[i] == None):
        traindata.release_date.loc[i] = aver_release

In [15]:
g_train_dictionary = texttodictionary(traindata.genres)
c_train_dictionary = texttodictionary(traindata.categories)
t_train_dictionary = texttodictionary(traindata.tags)
#storage the appeared words

In [16]:
traindata = texttoonehot(g_train_dictionary,traindata.genres,traindata)
traindata = texttoonehot(c_train_dictionary,traindata.categories,traindata)
traindata = texttoonehot(t_train_dictionary,traindata.tags,traindata)

In [17]:
f_traindata = traindata.drop(['id','playtime_forever','genres','categories','tags'], axis=1)
f_traindata = f_traindata.fillna(0)

In [22]:
rf = RandomForestRegressor(max_depth = 6,min_impurity_split = 15)

In [23]:
model = rf.fit(f_traindata,traindata.playtime_forever)

C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\tree\tree.py:297: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\tree\tree.py:297: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\tree\tree.py:297: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will c

In [24]:
for i in range(len(testdata.is_free)):
    testdata.is_free.loc[i] = is_freetonumber(testdata.is_free[i])
#doing the same data processing to testset

C:\Users\lenovo\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [25]:
for i in range(len(testdata.purchase_date)):
    testdata.purchase_date.loc[i] = stringtodatetime(testdata.purchase_date[i])
    testdata.release_date.loc[i] = stringtodatetime(testdata.release_date[i])
aver_purchase=0
aver_release=0
for i in range(len(testdata.release_date)):
    if(testdata.purchase_date[i] != None):
        aver_purchase += testdata.purchase_date[i]
    if(testdata.release_date[i] != None):
        aver_release += testdata.release_date[i]
aver_purchase = int(aver_purchase/len(testdata.release_date))
aver_release = int(aver_release/len(testdata.release_date))
for i in range(len(testdata.release_date)):
    if(testdata.purchase_date[i] == None):
        testdata.purchase_date.loc[i] = aver_purchase
    if(testdata.release_date[i] == None):
        testdata.release_date.loc[i] = aver_release

In [26]:
testdata = texttoonehot(g_train_dictionary,testdata.genres,testdata)
testdata = texttoonehot(c_train_dictionary,testdata.categories,testdata)
testdata = texttoonehot(t_train_dictionary,testdata.tags,testdata)

In [27]:
f_testdata = testdata.drop(['id','genres','categories','tags'], axis=1)

In [28]:
f_testdata = f_testdata.fillna(0)

In [29]:
y = model.predict(f_testdata)

In [30]:
save = {'id':testdata.id,'playtime_forever':y}

In [31]:
save = pd.DataFrame(save)

In [32]:
save.to_csv('E:/5001 individual/test1.csv',index=False,header=True)
#gennerate final submission